In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
from datetime import datetime

In [2]:
def plot_likelihood_vs_time_mpl(likelihoods, path, session_name, df_orig):
    plt.figure(figsize=(12, 6))
    for col in likelihoods.columns:
        # Using df_orig to get the body part name from row 0 as your code did
        plt.plot(likelihoods.index, likelihoods[col], label=str(df_orig.loc[0, col]), alpha=0.7)
    
    plt.title(f"Likelihoods over Time: {session_name}")
    plt.xlabel("Frame Index")
    plt.ylabel("Likelihood")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig(f'{path}/likelihood_vs_time_{session_name}.png')
    plt.close()

In [3]:
def plot_likelihood_dist_pooled_mpl(likelihoods, path, session_name):
    # Flatten and round
    vals = likelihoods.values.flatten().astype(float)
    vals = np.round(vals, 2)
    
    # Calculate distribution
    unique, counts = np.unique(vals, return_counts=True)
    probs = counts / counts.sum()

    plt.figure(figsize=(10, 5))
    plt.plot(unique, probs, marker='o', linestyle='-', markersize=2)
    plt.title(f"Pooled Confidence Distribution: {session_name}")
    plt.xlabel("Confidence Level")
    plt.ylabel("Probability")
    plt.grid(True, alpha=0.3)
    plt.savefig(f'{path}/likelihood_dist_pooled_{session_name}.png')
    plt.close()

In [4]:
def plot_likelihood_cumdist_pooled_mpl(likelihoods, path, session_name):
    vals = sorted(likelihoods.values.flatten().astype(float))
    # Cumulative distribution
    y = np.arange(1, len(vals) + 1) / len(vals)

    plt.figure(figsize=(10, 5))
    plt.plot(vals, y)
    plt.title(f"Cumulative Confidence: {session_name}")
    plt.xlabel("Confidence Level")
    plt.ylabel("Cumulative Probability")
    plt.grid(True, alpha=0.3)
    plt.savefig(f'{path}/likelihood_cumdist_pooled_{session_name}.png')
    plt.close()

In [5]:
def plot_likelihood_dist_each_mpl(likelihoods, path, session_name, df_orig):
    plt.figure(figsize=(12, 6))
    
    for col in likelihoods.columns:
        vals = np.round(likelihoods[col].astype(float).values, 2)
        unique, counts = np.unique(vals, return_counts=True)
        probs = counts / counts.sum()
        
        plt.plot(unique, probs, label=str(df_orig.loc[0, col]), alpha=0.8)

    plt.title(f"Distribution for Each Body Part: {session_name}")
    plt.xlabel("Confidence Level")
    plt.ylabel("Probability")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig(f'{path}/likelihood_dist_eachBodyPart_{session_name}.png')
    plt.close()

In [9]:
if __name__ == "__main__":
    # 1. Configuration
    path_to_model_videos_folder = "/home/hice1/saminnaji3/DLC/Model/Track-TA-2026-02-02/videos"
    path_to_evaluation_foler = "/home/hice1/saminnaji3/DLC/Evaluation"     
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 2. Get list of CSV files
    csv_dir_all = [d for d in os.listdir(path_to_model_videos_folder) if d.endswith('.csv')]
    
    if not csv_dir_all:
        print("No CSV files found in the directory!")

    for csv_dir in csv_dir_all:
        print(f"Processing: {csv_dir}")

        # Create unique folder for this specific CSV's figures
        # This prevents figures from different model iterations from overwriting each other
        suffix = csv_dir.split("_detector")[1].replace(".csv", "") if "_detector" in csv_dir else ""
        new_path = os.path.join(path_to_evaluation_foler, f"Figures_{timestamp}_{suffix}")
        os.makedirs(new_path, exist_ok=True)

        csv_path = os.path.join(path_to_model_videos_folder, csv_dir)
        
        # 3. Load and Clean Data
        df = pd.read_csv(csv_path)
        
        # Extract likelihood columns (every 3rd column starting from index 3)
        likelihood_indices = list(range(3, len(df.columns), 3))
        likelihoods = df.iloc[2:, likelihood_indices].copy()
        
        # Ensure data is numeric (DLC CSVs often read these as objects/strings)
        likelihoods = likelihoods.apply(pd.to_numeric, errors='coerce')
        
        # Use filename as session name
        session_name = csv_dir.replace(".csv", "")

        # 4. Generate Plots
        print(f"Generating Matplotlib figures in: {new_path}")
        
        plot_likelihood_vs_time_mpl(likelihoods, new_path, session_name, df)
        plot_likelihood_dist_pooled_mpl(likelihoods, new_path, session_name)
        plot_likelihood_cumdist_pooled_mpl(likelihoods, new_path, session_name)
        plot_likelihood_dist_each_mpl(likelihoods, new_path, session_name, df)

    print("All processing complete.")

Processing: SA17_20260114_croppedDLC_HrnetW48_TrackFeb2shuffle1_detector_best-130_snapshot_best-10.csv


/tmp/ipykernel_3092121/572018389.py:26: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


Generating Matplotlib figures in: /home/hice1/saminnaji3/DLC/Evaluation/Figures_20260204_080758__best-130_snapshot_best-10
All processing complete.
